# Bulk RNA Seq Analysis


The ultimate goal of this exercise and the accompanying R script is to generate figure 5 from this paper with two replicates each from the MBC and prePB groups https://www.nature.com/articles/s41375-021-01234-0

# Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Create directory for the course

In [ ]:
!mkdir /content/gdrive/MyDrive/colab
!mkdir /content/gdrive/MyDrive/colab/compgen

mkdir: cannot create directory ‘/content/gdrive/MyDrive/colab’: File exists
mkdir: cannot create directory ‘/content/gdrive/MyDrive/colab/compgen’: File exists


# Set up Anaconda and bioconda
You must run this every session, as well as subsequent installations.

In [ ]:
%cd /usr/local/

!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local/

import sys

sys.path.append("/usr/local/lib/python3.6/site-packages/")

!conda config --add channels defaults
!conda config --add channels bioconda
!conda config --add channels conda-forge
%cd /content/gdrive/MyDrive/colab

# Create and enter RNASeq directory


In [ ]:
%cd /content/gdrive/MyDrive/colab/compgen
#!rm -r /content/gdrive/MyDrive/colab/compgen/RNASeq
!mkdir /content/gdrive/MyDrive/colab/compgen/RNASeq
%cd /content/gdrive/MyDrive/colab/compgen/RNASeq


/content/gdrive/MyDrive/colab/compgen
/content/gdrive/MyDrive/colab/compgen/RNASeq


# Install fastq-dump

In [ ]:
%cd /
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
!gunzip sratoolkit.current-ubuntu64.tar.gz
!tar -xvf sratoolkit.current-ubuntu64.tar

# Download Fastq files

In [ ]:
%%bash
cd /content/gdrive/MyDrive/colab/compgen/RNASeq
LIST=$(cat accessions.txt)
mkdir fastq
cd /
rm -r fasterq.tmp.9cc98f13d0f0.49085/
for i in ${LIST}
    do
        ./sratoolkit.3.0.7-ubuntu64/bin/fasterq-dump ${i}
        echo ${i}' downloaded'
        mv /${i}_1.fastq /content/gdrive/MyDrive/colab/compgen/RNASeq/fastq/${i}_1.fastq
        mv /${i}_2.fastq /content/gdrive/MyDrive/colab/compgen/RNASeq/fastq/${i}_2.fastq
    done

# Install salmon

In [ ]:
%%bash
cd /
conda install -y salmon

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base conda




# Create config.txt and samples.txt
This code block creates a config file and a list of samples so that most of the rest of the script can be autonomous. This code chunk is written in python.

Config.txt will contain parameters that will be loaded into a subsequent bash script and used to complete variable names in order to download gencode files and properly format file names.
Samples.txt will contain a list of your sample names.

Instructions:
- Edit species to reflect human or mouse.
- Edit version to reflect the current gencode release. Do NOT include the 'M' prefix to the mouse release. Note: you can check the current releases here https://www.gencodegenes.org/.
- Edit the fastqExtension. If your fastq file names have a ton of junk in the middle of them, this refers to that junk. Example below.
- Edit the list of your samples. Example below.

The fastqExtension and samples list are used in the salmon quant code blocks. Check them to see more about why they are formatted the way they are.

Example:
Let's say you have three paired end samples to quantify: BL, H6, and L24. Your list of fastqs looks like this:
- BLa739_9747_AHFD7D_1.fastq
- BLa739_9747_AHFD7D_2.fastq
- H6a739_9747_AHFD7D_1.fastq
- H6a739_9747_AHFD7D_2.fastq
- L24a739_9747_AHFD7D_1.fastq
- L24a739_9747_AHFD7D_2.fastq

Your variables in the script below would then be:

fastqExtension = 'a739_9747_AHDFD7D_'

samples = {'BL', 'H6', 'L6']

Note: If you have paired end reads, do NOT list each fastq separately! List each SAMPLE separately.

In [ ]:
%cd /content/gdrive/MyDrive/colab/compgen/RNASeq
species = 'humn'
gencodeVersion = 44
fastqExtension = '_CKDL230019755-1A_H75TNDSX7_L2_'
samples = ['sixA', 'sixB', 'tfourA', 'tfourB', 'zeroA', 'zeroB']

if species == 'human':
    speciesfull = 'human'
    specieslow = 'h'
    speciesup = ''
    release = 38
elif species == 'mouse':
    speciesfull = 'mouse'
    specieslow = 'm'
    speciesup = 'M'
    release = 39
else:
    print('Error: you formatted your species entry incorrectly')

config = open("config.txt", "w")
configelements = [fastqExtension, gencodeVersion, speciesfull, specieslow, speciesup, release]
for line in configelements:
    config.write("%s\n" % line)
config.close()

samps = open("samples.txt", "w")
for line in samples:
    samps.write("%s\n" % line)
samps.close()
print('Done')

/content/gdrive/MyDrive/colab/compgen/RNASeq
Error: you formatted your species entry incorrectly
Done


# Download gencode files, generate decoys.txt + gene_map.csv, and run salmon index
If you did things right and I did things right, you should be able to just run this code block if you're working with mouse or human data. Good luck!

That being said, you MUST verify that each step of this script actually worked before the subsequent steps will run correctly. If you just hit "run" and hope for the best, you may not catch your error until way downstream when all your files are empty.

If you're NOT working with mouse or human data, you need to edit the wget lines to whatever will download your transcript and primary assembly files, as well as any subsequent file paths that point to the downloaded files.

In [ ]:
%%bash
%cd /content/gdrive/MyDrive/colab/compgen/RNASeq
VERS=$(sed -n '2p' config.txt)
SPECIESFULL=$(sed -n '3p' config.txt)
SPECIESlow=$(sed -n '4p' config.txt)
SPECIESup=$(sed -n '5p' config.txt)
RELEASE=$(sed -n '6p' config.txt)

#download needed gencode files
wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_${SPECIESFULL}/release_${SPECIESup}${VERS}/gencode.v${SPECIESup}${VERS}.transcripts.fa.gz
wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_${SPECIESFULL}/release_${SPECIESup}${VERS}/GRC${SPECIESlow}${RELEASE}.primary_assembly.genome.fa.gz

#generate decoys.txt from the transcript and primary assembly files
grep "^>" <(gunzip -c GRC${SPECIESlow}${RELEASE}.primary_assembly.genome.fa.gz) | cut -d " " -f 1 > decoys.txt
sed -i.bak -e 's/>//g' decoys.txt
cat gencode.v${SPECIESup}${VERS}.transcripts.fa.gz GRC${SPECIESlow}${RELEASE}.primary_assembly.genome.fa.gz > gentrome.fa.gz

#create gene_map.csv
gunzip gencode.v${SPECIESup}${VERS}.transcripts.fa.gz
echo ${SPECIESlow}
if [[ ${SPECIESlow} == "h" ]]; then
    echo 'human'
    grep -P -o 'ENST\d{11}' gencode.v${SPECIESup}${VERS}.transcripts.fa > enst.txt
    grep -P -o 'ENSG\d{11}' gencode.v${SPECIESup}${VERS}.transcripts.fa > ensg.txt
else
    echo 'mouse'
    grep -P -o 'ENSMUST\d{11}' gencode.v${SPECIESup}${VERS}.transcripts.fa > enst.txt
    grep -P -o 'ENSMUSG\d{11}' gencode.v${SPECIESup}${VERS}.transcripts.fa > ensg.txt
fi
paste -d ',' enst.txt ensg.txt > gene_map.csv
gzip gencode.v${SPECIESup}${VERS}.transcripts.fa

#run salmon indexer
salmon index -t gentrome.fa.gz -d decoys.txt -p 12 -i salmon_index --gencode
echo 'Done'


Process is terminated.


# Run Salmon quant on paired end reads
Run this code block if you have paired end reads.

Validate it step by step FIRST so that you don't end up with a dozen unzipped fastqs.

You may have to reformat the way the fastq's are named based on how your fastq names are formatted. Use echo statements to debug.

In [ ]:
%%bash
conda activate salmon
LIST=$(cat samples.txt)
FQEXT=$(sed -n '1p' config.txt)
for i in ${LIST}
    do
        gunzip fastq/${i}${FQEXT}1.fq.gz
        gunzip fastq/${i}${FQEXT}2.fq.gz
        salmon quant -i salmon_index -l A -1 fastq/${i}${FQEXT}1.fq -2 fastq/${i}${FQEXT}2.fq --validateMappings -o ${i}
        gzip fastq/${i}${FQEXT}1.fq
        gzip fastq/${i}${FQEXT}2.fq
    done

# Run salmon quant on single end reads
Run this code block if you have single end reads.

Validate it step by step FIRST so that you don't end up with a dozen unzipped fastqs.

You may have to reformat the way the fastq's are named based on how your fastq names are formatted. Use echo statements to debug.

In [ ]:
%%bash
conda activate salmon
LIST=$(cat samples.txt)
FQEXT=$(sed -n '1p' config.txt)
for i in ${LIST}
    do
        gunzip fastq/${i}${FQEXT}.fq.gz
        salmon quant -i salmon_index -l A -r fastq/${i}${FQEXT}.fq --validateMappings -o ${i}
        gzip fastq/${i}${FQEXT}.fq
    done

fastq/sixA_CKDL230019755-1A_H75TNDSX7_L2_2.fq
Process is interrupted.


# Generate SRARunTable.txt files
DeSeq2 will eventually rearrange your data so that it is alphabetical and then treats whatever comes first as the control. Make sure your controls are listed FIRST and are the first of your samples alphabetically. You may need to edit the names of your salmon outputs to reflect this. Edit these lists in the code block below according to the following:

- samples: This should be a list of your samples. Unless you need to make changes for alphabetization, this should match your sample list from the first code block.
- treatments: Every value is either "control" or "treatment" in this list. List positions correspond to sample list positions.
- conditions: Each value is an experiment-specific treatment label. These are used to group samples by condition.
- cell_line: Each value is the cell line that the experiment sample was carried out in.
- files: This should be your ONLY list of a different length that does NOT correspond to positions in the other lists. This should just be a list of conditions that exist in your experiment, excluding "control." However you format this should match what you tell R in subsequent analysis.

For an example, see the start to the code block below.

In [ ]:
samples = ['controlA','controlB','NonionA','NonionB','PBSA','PBSB','SM102eLNPA','SM102eLNPB','SM102RNAA','SM102RNAB','TCLeLNPA','TCLeLNPB','TCLRNAA','TCLRNAB']
treatments = ["control", "control", "treatment", "treatment", "treatment", "treatment", "treatment", "treatment","treatment","treatment","treatment","treatment","treatment","treatment"]
files = ["nonion","PBS","SM102eLNP","SM102RNA","TCLeLNP","TCLRNA"]
conditions = ["control", "control","nonion","nonion","PBS","PBS","SM102eLNP","SM102eLNP","SM102RNA","SM102RNA","TCLeLNP","TCLeLNP","TCLRNA","TCLRNA"]
cell_line = ['mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse']


SRT = open("SraRunTableall.txt", "w")
SRT.write("%s\n" % "sample name, treatment, cell_line")
for i in range(len(samples)):
    SRT.write(samples[i])
    SRT.write(', ')
    SRT.write(treatments[i])
    SRT.write(', ')
    SRT.write("%s\n" % cell_line[i])
SRT.close()

for j in range(len(files)):
    filename = "SraRunTable"+files[j]+".txt"
    SRT = open(filename, "w")
    SRT.write("%s\n" % "sample name, treatment, cell_line")
    for i in range(len(samples)):
        if conditions[i] == 'control' or conditions[i] == files[j]:
            SRT.write(samples[i])
            SRT.write(', ')
            SRT.write(treatments[i])
            SRT.write(', ')
            SRT.write("%s\n" % cell_line[i])
    SRT.close()

print("Done")

Done
